In [35]:
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
import time

import numpy as np
import pandas as pd

options = Options()

#Comment this out if you want to see the browser, mainly usefull for debugging
options.add_argument("--headless")

pd.set_option('display.max_rows', 1000)

driver = webdriver.Chrome(options=options)

In [36]:
# Parses the odds and lines from the stings from the list
def parseInfo(info, default_odds):
    if "(" not in info:
        return int(info), default_odds
    else:
        seg = info.split("(", 1)

        # Extract the line and the odds string from seg
        l = int(seg[0].strip())  # Convert the number to an integer
        o = seg[1].strip(" )") 
        return l, o

In [37]:
def get_prop_lines(select, value):
    #Selects the prop passed as 'value' here from dropdown menu
    select.select_by_value(value)
    driver.implicitly_wait(10)

    #Initializes dataframe to store results
    result = pd.DataFrame(columns=["name", "team", "lines", "odds"])

    #fetches all games for a given prop and store them in 'games'
    games_cont = driver.find_element(By.CLASS_NAME, "wrapper")
    games = games_cont.find_elements(By.TAG_NAME, "app-game")

    time.sleep(3) # Wait for games to load
    for game in games:
        game.click()

        game_content = WebDriverWait(game, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".content")))

        #Stores each individual prop bet in 'players'
        players = game_content.find_elements(By.CLASS_NAME, "player-group")
        # print(f"Players: {len(players)}") (For debugging)

        for player in players:
            # Gets attributes for each individual prop bet and scrapes their info
            name = player.find_element(By.TAG_NAME, "b").text
            team = player.find_element(By.CLASS_NAME, "ss-team-names").text
            lines = player.find_elements(By.TAG_NAME, "option")
            default_odds = player.find_element(By.TAG_NAME, "button").text

            #Iterates through all the lines for each prop bet and adds them to the dataframe as a new row for each different line        
            for line in lines:
                info = line.text
                l, o = parseInfo(info, default_odds)
                wager = pd.DataFrame([{"name": name, "team": team, "lines": l, "odds": o}])
                frames = [result, wager]
                result = pd.concat(frames, ignore_index=True)
                # print(name, team, l, o) (For debugging)

    return result

In [38]:
#Source function to run the scraper
def bovScrape_league_props(url):
    driver.get(url)

    #Navigates to part of page where the prop bets are, enters the iframe and accesses the dropdown menu
    wait = WebDriverWait(driver, 10)
    league_box = wait.until(EC.presence_of_element_located((By.ID, "playerPropsLeagueFrame")))

    league_frame = league_box.find_element(By.TAG_NAME, "iframe")
    driver.switch_to.frame(league_frame)

    dropdown = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "select")))
    select = Select(dropdown)

    time.sleep(5) #Need to wait for the games to load properly
    shots_lines = get_prop_lines(select, "Shots")
    shots_on_goal_lines = get_prop_lines(select, "Shots on Goal")


    return shots_lines, shots_on_goal_lines
    

In [39]:
mx_url = 'https://www.bovada.lv/sports/soccer/north-america/mexico/liga-mx-apertura'
mls_url = 'https://www.bovada.lv/sports/soccer/north-america/united-states/mls'
# Should be able to add any other league url here and it should work

shots, sogs = bovScrape_league_props(mls_url)
shots

,name,team,lines,odds
0,Alexandru Matan,CLB,2,+197
1,Alexandru Matan,CLB,3,+630
2,Alexandru Matan,CLB,4,+2045
3,Alexandru Matan,CLB,5,+4000
4,Christian Ramirez,CLB,2,-173
5,Christian Ramirez,CLB,3,+142
6,Christian Ramirez,CLB,4,+359
7,Christian Ramirez,CLB,5,+1010
8,Christian Ramirez,CLB,6,+2670
9,Cucho Hernandez,CLB,3,-371


In [40]:
sogs

,name,team,lines,odds
0,Alexandru Matan,CLB,1,+300
1,Alexandru Matan,CLB,2,+1900
2,Alexandru Matan,CLB,3,+4900
3,Alexandru Matan,CLB,4,+12500
4,Alexandru Matan,CLB,5,+27500
5,Christian Ramirez,CLB,1,-167
6,Christian Ramirez,CLB,2,+300
7,Christian Ramirez,CLB,3,+1100
8,Christian Ramirez,CLB,4,+2900
9,Christian Ramirez,CLB,5,+7400
